In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (8.0, 10.0)
mpl.rcParams.update({'errorbar.capsize': 8})

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png')

In [ ]:
'''
Thompson Sampling
''' 
# steps and size of the problem
T = 10000
K = 5

# probabilities of reward
probs = [1.0/6, 1.0/2, 2.0/3, 3.0/4, 5.0/6]

# initial posterior distribution parmeters : successful and failed trails
succs = [0 for i in range(K)]
fails = [0 for i in range(K)]

for t in range(T):
    # calculate posteriors likelihood for all arms
    posteriors = [np.random.beta(succs[i] + 1, fails[i] + 1) for i in range(K)]

    # choose the arm with highest posterior likelihood to play for this step
    i_chosen = np.argmax(posteriors)
    
    # get the probablity of reward for the chosen arm
    prob = probs[i_chosen]
    
    # observe the reward
    reward = np.random.binomial(1,prob)
    
    # upadte the posterior distribution parmeters based on reward
    if reward == 1:
        succs[i_chosen] += 1 # if reward is 1, increase successful trails by 1
    else: 
        fails[i_chosen] += 1  # if reward is 0, increase failed trails by 1

In [ ]:
def betaVariance(alpha, beta):
    return alpha * beta * 1.0/ ((alpha + beta)**2 * (alpha + beta + 1))

In [ ]:
'''
Thompson Sampling Time Storing
''' 
# steps and size of the problem
T = 10000
K = 5

# probabilities of reward
probs = [1.0/6, 1.0/2, 2.0/3, 3.0/4, 5.0/6]

# initial posterior distribution parmeters : successful and failed trails
succs = [0 for i in range(K)]
fails = [0 for i in range(K)]

# list to store info
choice_list = []
avg_reward_list = []
total_reward = 0

# reward and conf level per arm
total_pulled_per_arm = [1e-32 for i in range(K)]
total_reward_per_arm = [0 for i in range(K)]

Nat = []
reward_per_arm_list = []
variance_per_arm_list = []

for t in range(T):
    # calculate posteriors likelihood for all arms
    posteriors = [np.random.beta(succs[i] + 1, fails[i] + 1) for i in range(K)]
    
    # choose the arm with highest posterior likelihood to play for this step
    i_chosen = np.argmax(posteriors)
    choice_list.append(i_chosen)
    
    # get the probablity of reward for the chosen arm
    prob = probs[i_chosen]
    
    # observe the reward
    reward = np.random.binomial(1,prob)
    total_reward += reward
    if t == 0:
        avg_reward_list.append(np.nan)
    else:
        avg_reward_list.append(total_reward * 1.0 / t)
        
    # update the values per arm
    total_pulled_per_arm[i_chosen] += 1
    total_reward_per_arm[i_chosen] += reward
    
    Nat.append(total_pulled_per_arm[:])
    reward_per_arm_list.append([total_reward_per_arm[k] * 1.0 / total_pulled_per_arm[k] for k in range(K)])
    variance_per_arm_list.append([betaVariance(succs[k] + 1, fails[k] + 1) for k in range(K)])

    # upadte the posterior distribution parmeters based on reward
    if reward == 1:
        succs[i_chosen] += 1 # if reward is 1, increase successful trails by 1
    else: 
        fails[i_chosen] += 1  # if reward is 0, increase failed trails by 1        

In [ ]:
t1 = 1000
Rt1 = reward_per_arm_list[t1]
cft1 = [np.sqrt(v) for v in variance_arm_list[t1]]
print(Rt1)
print(cft1)
print(total_pulled_per_arm)

In [ ]:
# plot expected regret with time
mu_star = max(probs)
ts = [i for i in range(T)]
avg_regret_list = [mu_star - avg_reward_list[t] for t in ts]

# avg. regret
plt.figure(figsize=(9,6),dpi=600)
plt.plot(ts, avg_regret_list, ".-", linewidth = 1)
plt.xlabel("Time Step", fontsize = 16)
plt.ylabel("Average Regret Value", fontsize = 16)
plt.title("Average Regret vs. Time Step", fontsize = 16)
plt.xlim([0,1000])
# plt.ylim([-0.2,0.3])
plt.tick_params(axis='both',labelsize=16)

In [ ]:
# plot confidence interval figures
t1 = 10
Rt1 = reward_per_arm_list[t1]
cft1 = [np.sqrt(v) for v in variance_arm_list[t1]]

t2 = 50
Rt2 = reward_per_arm_list[t2]
cft2 = [np.sqrt(v) for v in variance_arm_list[t2]]

t3 = 100
Rt3 = reward_per_arm_list[t3]
cft3 = [np.sqrt(v) for v in variance_arm_list[t3]]

t4 = 5000
Rt4 = reward_per_arm_list[t4]
cft4 = [np.sqrt(v) for v in variance_arm_list[t4]]

arms = [a for a in range(K)]
arms_label = [a+1 for a in range(K)]
plt.figure(figsize=(18,12),dpi=600)

plt.subplot(221)
plt.errorbar(arms, Rt1, yerr=cft1, fmt=".") 
plt.plot(arms, Rt1, "^", color="blue", markersize=12)
plt.plot(arms, probs, "o", markersize=12, markeredgewidth=2,markeredgecolor='red', markerfacecolor='None')
plt.xlabel("Arms", fontsize = 16)
plt.ylabel("Reward", fontsize = 16)
plt.legend(["estimated","true"], fontsize=16,loc="lower right")
plt.title("Confidence Interval Figure at t = " + str(t1), fontsize = 16)
plt.xticks(arms, arms_label)
plt.tick_params(axis='both',labelsize=16)
# plt.xlim([0.1,3])
# plt.ylim([-0.2,0.3])

plt.subplot(222)
plt.errorbar(arms, Rt2, yerr=cft2, fmt=".") 
plt.plot(arms, Rt2, "^", color="blue", markersize=12)
plt.plot(arms, probs, "o", markersize=12, markeredgewidth=2,markeredgecolor='red', markerfacecolor='None')
plt.xlabel("Arms", fontsize = 16)
plt.ylabel("Reward", fontsize = 16)
plt.legend(["estimated","true"], fontsize=16,loc="lower right")
plt.title("Confidence Interval Figure at t = " + str(t2), fontsize = 16)
plt.xticks(arms, arms_label)
plt.tick_params(axis='both',labelsize=16)

plt.subplot(223)
plt.errorbar(arms, Rt3, yerr=cft3, fmt=".") 
plt.plot(arms, Rt3, "^", color="blue", markersize=12)
plt.plot(arms, probs, "o", markersize=12, markeredgewidth=2,markeredgecolor='red', markerfacecolor='None')
plt.xlabel("Arms", fontsize = 16)
plt.ylabel("Reward", fontsize = 16)
plt.legend(["estimated","true"], fontsize=16,loc="lower right")
plt.title("Confidence Interval Figure at t = " + str(t3), fontsize = 16)
plt.xticks(arms, arms_label)
plt.tick_params(axis='both',labelsize=16)

plt.subplot(224)
plt.errorbar(arms, Rt4, yerr=cft4, fmt=".") 
plt.plot(arms, Rt4, "^", color="blue", markersize=12)
plt.plot(arms, probs, "o", markersize=12, markeredgewidth=2,markeredgecolor='red', markerfacecolor='None')
plt.xlabel("Arms", fontsize = 16)
plt.ylabel("Reward", fontsize = 16)
plt.legend(["estimated","true"], fontsize=16,loc="lower right")
plt.title("Confidence Interval Figure at t = " + str(t4), fontsize = 16)
plt.xticks(arms, arms_label)
plt.tick_params(axis='both',labelsize=16)

In [ ]:
# make a plot the fraction of time (up to time t) for which the algorithm pulled arm a
ts = [t for t in range(1,T)]
Nadt = []
for k in range(K):
    Nadt.append([Nat[t][k]*1.0 / t for t in ts])

plt.figure(figsize=(9,6),dpi=600)
for k in range(K):
    plt.plot(ts, Nadt[k], "-", linewidth = 1)
plt.xlabel("Time Step", fontsize = 16)
plt.ylabel("Fraction of Time", fontsize = 16)
plt.title("Fraction of Time the Algorithm Pulled Arm a", fontsize = 16)
plt.legend(["arm 1","arm 2","arm 3","arm 4", "arm 5"], fontsize = 16)
plt.xlim([0,1000])
plt.ylim([0,1])
plt.tick_params(axis='both',labelsize=16)    

In [ ]:
# make a plot the fraction of time (up to time t) for which the algorithm pulled arm a
ts = [t for t in range(1,T)]
k = 4
Nadt5 = [Nat[t][k]*1.0 / t for t in ts]

plt.figure(figsize=(9,6),dpi=600)
plt.plot(ts, Nadt5, "-", linewidth = 1)

plt.xlabel("Time Step", fontsize = 16)
plt.ylabel("Fraction of Time", fontsize = 16)
plt.title("Fraction of Time the Algorithm Pulled Arm a 5", fontsize = 16)
plt.xlim([0,5000])
plt.axhline(y = 0.95,linestyle=":",linewidth=1, color="r")
plt.ylim([0,1])
plt.tick_params(axis='both',labelsize=16)   

In [ ]:
[i for i in range(len(Nadt5)) if Nadt5[i] >= 0.95]